In [33]:
import duckdb
import pandas as pd
import ast

In [34]:
# Connect to DuckDB (creates a file-based DB)
con = duckdb.connect("/home/leloc/Document/USTH/Thesis/Data/translation.db")

print(con.execute("SELECT * FROM translations LIMIT 5").fetchdf())

                                  thai  \
0                    - รร.แกรนด์โฮเต็ล   
1                         - สายไม่ว่าง   
2                         นั่นใครพูดคะ   
3  ฉันจะโอนสายคุณ ไปที่แผนกรูมเซอร์วิส   
4                              ฮัลโหลๆ   

                                              viet  \
0                               - Khách sạn Grand.   
1                                 - Đường dây bận.   
2                                          Ai đấy?   
3  Tôi có thể kết nối quý khách với dịch vụ phòng.   
4                                            A lô.   

                                      thai_input_ids  \
0  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
1  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
2  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
3  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
4  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   

                                 thai_attention_mask  \
0  [128, 4, 149, 228, 1, 0, 0

In [35]:
tables = con.execute("SHOW TABLES").fetchall()
print(tables)

[('tokenized_batches',), ('translations',)]


In [36]:
df = con.execute("SELECT thai_input_ids, vi_input_ids FROM translations LIMIT 10").fetchdf()
print(df.head())

                                      thai_input_ids  \
0  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
1  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
2  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
3  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   
4  [128, 4, 149, 228, 1, 0, 0, 0, 0, 0, 0, 140, 2...   

                                        vi_input_ids  
0  [128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...  
1  [128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...  
2  [128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...  
3  [128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...  
4  [128, 4, 149, 196, 1, 0, 0, 0, 0, 0, 0, 140, 2...  


In [37]:
print(df.dtypes)


thai_input_ids    object
vi_input_ids      object
dtype: object


In [38]:
import pickle

# Hàm để giải mã pickle (từ byte sang list token)
def decode_pickle_data(x):
    try:
        return pickle.loads(x)
    except Exception as e:
        print("Lỗi giải mã pickle:", e)
        return []

# Giải mã pickle thành list
df["thai_input_ids"] = df["thai_input_ids"].apply(decode_pickle_data)
df["vi_input_ids"] = df["vi_input_ids"].apply(decode_pickle_data)

# Định nghĩa padding token (thường là 0 và 1)
PADDING_IDS = {0, 1}

# Tính độ dài thật sự (không tính padding)
df["thai_true_len"] = df["thai_input_ids"].apply(lambda x: len([t for t in x if t not in PADDING_IDS]))
df["viet_true_len"] = df["vi_input_ids"].apply(lambda x: len([t for t in x if t not in PADDING_IDS]))

# Tính độ dài toàn bộ chuỗi token (bao gồm padding)
df['thai_token_len'] = df['thai_input_ids'].apply(len)
df['viet_token_len'] = df['vi_input_ids'].apply(len)

# In ra so sánh
print(df[['thai_token_len', 'thai_true_len', 'viet_token_len', 'viet_true_len']])


   thai_token_len  thai_true_len  viet_token_len  viet_true_len
0              43             15              39              7
1              43              7              39              8
2              43              6              39              5
3              43             17              39             14
4              43              7              39              5
5              43             32              39             21
6              43             19              39             21
7              43             12              39             12
8              43             22              39             16
9              43              7              39             10


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

InvalidInputException: Invalid Input Error: Python exception occurred while executing the UDF: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  C:\Users\Acer\AppData\Local\Temp\ipykernel_22736\2889407501.py(9): hf_token_len
  C:\Users\Acer\AppData\Local\Temp\ipykernel_22736\2889407501.py(38): <module>
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\interactiveshell.py(3549): run_code
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\interactiveshell.py(3489): run_ast_nodes
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\interactiveshell.py(3306): run_cell_async
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\async_helpers.py(128): _pseudo_sync_runner
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\interactiveshell.py(3102): _run_cell
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages\IPython\core\interactiveshell.py(3047): run_cell
  c:\Users\Acer\anaconda3\envs\thesis\Lib\site-packages